<a href="https://colab.research.google.com/github/guanyu1127/Programming-Language/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 1. 安裝所有必要的套件
!pip install folium pandas gspread gspread-dataframe gradio

# 2. 匯入所有必要的函式庫
import folium
import pandas as pd
import gradio as gr
import gspread
from google.colab import auth
from google.auth import default
from gspread_dataframe import get_as_dataframe

# 3. 設定 Google Sheet 的常數
# 新的東京餐廳 Google Sheet 網址
SHEET_URL = "https://docs.google.com/spreadsheets/d/1xUtKgD13TWQ94cnspBCublp-_LZm_knEor2Kx_uf8u4/edit?usp=sharing"
# 請確認您的 Google Sheet 中，分頁標籤的名稱是否為 "工作表1"。如果不是，請修改這裡。
WORKSHEET_NAME = "工作表1"

# 4. 定義一個函數來生成東京地圖
def create_tokyo_restaurant_map():
    # 步驟 4.1: Google 驗證與 gspread 授權
    try:
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)
    except Exception as e:
        return f"Google 驗證失敗: {e}"

    # 步驟 4.2: 讀取 Google Sheet 資料
    try:
        gsheets = gc.open_by_url(SHEET_URL)
        # 嘗試讀取指定的工作表
        worksheet = gsheets.worksheet(WORKSHEET_NAME)
        # 將資料載入為 DataFrame
        df_restaurants = get_as_dataframe(worksheet, evaluate_formulas=True)
        # 移除完全空白的行 (如果有的話)
        df_restaurants = df_restaurants.dropna(how='all')
        # 確保 lat 和 lng 欄位是數值型態，無法轉換的變成 NaN
        df_restaurants['lat'] = pd.to_numeric(df_restaurants['lat'], errors='coerce')
        df_restaurants['lng'] = pd.to_numeric(df_restaurants['lng'], errors='coerce')
        # 移除沒有經緯度的資料
        df_restaurants = df_restaurants.dropna(subset=['lat', 'lng'])

    except gspread.exceptions.WorksheetNotFound:
        return f"錯誤：找不到名稱為 '{WORKSHEET_NAME}' 的工作表。請確認您的 Google Sheet 分頁名稱是否正確。"
    except Exception as e:
        return f"讀取 Google Sheet 失敗: {e}"

    # 步驟 4.3: 建立 Folium 地圖
    # 設定地圖中心點為東京 (大約在新宿附近)，縮放層級設為 13
    lat_center_tokyo = 35.69
    lng_center_tokyo = 139.70
    m = folium.Map(location=[lat_center_tokyo, lng_center_tokyo], zoom_start=13)

    # 步驟 4.4: 遍歷 DataFrame 並在地圖上添加標記
    for _, row in df_restaurants.iterrows():
        # 客製化彈出視窗(Popup)內容
        # 使用 .get() 來避免如果欄位名稱稍微不同時產生錯誤，提供預設空字串
        name = row.get('店家名稱', '未命名店家')
        category = row.get('類別/招牌品項', '')
        location_ref = row.get('參考地點 (捷運站/商圈)', '')
        note = row.get('備註/特色', '')

        popup_html = f"""
        <div style="width:250px">
            <h4><b>{name}</b></h4>
            <p><b>類別:</b> {category}</p>
            <p><b>地點:</b> {location_ref}</p>
            <p style="font-size:0.9em; color:gray;"><i>{note}</i></p>
        </div>
        """

        folium.Marker(
            location=[row['lat'], row['lng']],
            tooltip=name, # 滑鼠懸停時顯示店名
            popup=folium.Popup(popup_html, max_width=300),
            # 使用紅色的餐具圖示來區別
            icon=folium.Icon(icon="cutlery", prefix='fa', color="red")
        ).add_to(m)

    # 步驟 4.5: 返回地圖的 HTML
    return m._repr_html_()

# 5. 建立並啟動 Gradio 介面
print("正在啟動東京美食地圖 Gradio 介面...")
iface = gr.Interface(
    fn=create_tokyo_restaurant_map,
    inputs=None,
    outputs=gr.HTML(),
    title="東京在地美食地圖",
    description="探索東京的精選在地美食！資料來源：Google Sheet"
)

# 啟動介面
iface.launch(debug=True)

正在啟動東京美食地圖 Gradio 介面...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://eb58bca732c589b187.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://eb58bca732c589b187.gradio.live
